In [ ]:
# Check if the easydiffraction library is installed.
# If not, install it including the 'visualization' extras.
# This is needed, e.g., when running this as a notebook via Google Colab or
# Jupyter Notebook in an environment where the library is not pre-installed.
import builtins
import importlib.util

if hasattr(builtins, '__IPYTHON__'):
    if importlib.util.find_spec('easydiffraction') is None:
        !pip install 'easydiffraction[visualization]'


# Structure Refinement: Si, SEPD

This example demonstrates a Rietveld refinement of Si crystal structure using
time-of-flight neutron powder diffraction data from SEPD at Argonne.

## Import Library

In [ ]:
from easydiffraction import Experiment
from easydiffraction import Project
from easydiffraction import SampleModel
from easydiffraction import download_from_repository

## Define Sample Model

This section shows how to add sample models and modify their parameters.

#### Create Sample Model

In [ ]:
model = SampleModel('si')

#### Set Space Group

In [ ]:
model.space_group.name_h_m = 'F d -3 m'
model.space_group.it_coordinate_system_code = '2'

#### Set Unit Cell

In [ ]:
model.cell.length_a = 5.431

#### Set Atom Sites

In [ ]:
model.atom_sites.add('Si', 'Si', 0.125, 0.125, 0.125, b_iso=0.5)

## Define Experiment

This section shows how to add experiments, configure their parameters, and
link the sample models defined in the previous step.

#### Download Measured Data

In [ ]:
download_from_repository('sepd_si.xye', destination='data')

#### Create Experiment

In [ ]:
expt = Experiment('sepd', beam_mode='time-of-flight', data_path='data/sepd_si.xye')

#### Set Instrument

In [ ]:
expt.instrument.setup_twotheta_bank = 144.845
expt.instrument.calib_d_to_tof_offset = 0.0
expt.instrument.calib_d_to_tof_linear = 7476.91
expt.instrument.calib_d_to_tof_quad = -1.54

#### Set Peak Profile

In [ ]:
expt.peak_profile_type = 'pseudo-voigt * ikeda-carpenter'
expt.peak.broad_gauss_sigma_0 = 3.0
expt.peak.broad_gauss_sigma_1 = 40.0
expt.peak.broad_gauss_sigma_2 = 2.0
expt.peak.broad_mix_beta_0 = 0.04221
expt.peak.broad_mix_beta_1 = 0.00946

#### Set Peak Asymmetry

In [ ]:
expt.peak.asym_alpha_0 = 0.0
expt.peak.asym_alpha_1 = 0.5971

#### Set Background

In [ ]:
expt.background_type = 'line-segment'
for x in range(0, 35000, 5000):
    expt.background.add(x=x, y=200)

#### Set Linked Phases

In [ ]:
expt.linked_phases.add('si', scale=10.0)

## Define Project

The project object is used to manage the sample model, experiment, and
analysis.

#### Create Project

In [ ]:
project = Project()

#### Set Plotting Engine

In [ ]:
project.plotter.engine = 'plotly'

#### Add Sample Model

In [ ]:
project.sample_models.add(model)

#### Add Experiment

In [ ]:
project.experiments.add(expt)

## Perform Analysis

This section shows the analysis process, including how to set up
calculation and fitting engines.

#### Set Calculator

In [ ]:
project.analysis.current_calculator = 'cryspy'

#### Set Minimizer

In [ ]:
project.analysis.current_minimizer = 'lmfit (leastsq)'

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='sepd', show_residual=True)
project.plot_meas_vs_calc(expt_name='sepd', x_min=23200, x_max=23700, show_residual=True)

### Perform Fit 1/5

Set parameters to be refined.

In [ ]:
model.cell.length_a.free = True

expt.linked_phases['si'].scale.free = True
expt.instrument.calib_d_to_tof_offset.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='sepd', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='sepd', x_min=23200, x_max=23700, show_residual=True)

### Perform Fit 2/5

Set more parameters to be refined.

In [ ]:
for point in expt.background:
    point.y.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='sepd', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='sepd', x_min=23200, x_max=23700, show_residual=True)

### Perform Fit 3/5

Fix background points.

In [ ]:
for point in expt.background:
    point.y.free = False

Set more parameters to be refined.

In [ ]:
expt.peak.broad_gauss_sigma_0.free = True
expt.peak.broad_gauss_sigma_1.free = True
expt.peak.broad_gauss_sigma_2.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='sepd', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='sepd', x_min=23200, x_max=23700, show_residual=True)

### Perform Fit 4/5

Set more parameters to be refined.

In [ ]:
model.atom_sites['Si'].b_iso.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='sepd', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='sepd', x_min=23200, x_max=23700, show_residual=True)